# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [208]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
      
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    
    return L, u

In [209]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [210]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [211]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  1.110e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -2.819e-16  0.000e+00  0.000e+00  8.080e-04  1.902e-03]
 [ 0.000e+00  3.369e-16  0.000e+00 -1.541e-18  2.168e-19 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16  2.776e-16 -2.776e-16  5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [212]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [213]:
np.linalg.matrix_rank(a1)

6

In [214]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in true_divide
  app.launch_new_instance()
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in matmul
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in matmul
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [215]:
N = a.shape[0]
N1 = a1.shape[0]
с = False
c1 = False
det = [[],[]]

### <GET DETERMINATES>
for i in range (N):
    det[0].append(np.linalg.det(a[0:i+1, 0:i+1]))
    det[1].append(np.linalg.det(a1[0:i+1, 0:i+1]))  

### <CHECK OF REQUIRMENT>
for i in range(len(det)):
    if i == 0:
        a0 = ''
    else:
        a0 = 1
        
    clause = 0
    for j in range(a.shape[0]):
        if det[i][j] == 0:
            clause += 1
            break
    
    if clause > 0:
        print('Requirment isn`t satysfied for matrix a', a0, sep = '')
    else:
        print('Requirment is satysfied for matrix a', a0, sep = '')

### <SHOW ALL LEADING MINORS>
### for i in range (N):
###     print(a[0:i+1, 0:i+1])
###     print(a1[0:i+1, 0:i+1])

print('')
print('Determinates of a and a1 in order')
print(det[0])
print(det[1])

Requirment is satysfied for matrix a
Requirment isn`t satysfied for matrix a1

Determinates of a and a1 in order
[3.0000000000000004, -3.375, -0.8859990277102583, 0.01946700103200583, 1.5728974209738795e-05, -2.4923994635008296e-10]
[3.0000000000000004, 0.0, -8.03305785123967, -0.49351010616247487, 0.0009030145201489381, 2.3183841674291354e-08]


# Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [216]:
def lup_reconstruct(L, u, P):
    return np.matrix.transpose(P) @ L @ u

def diy_lu_modified(a):

    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    P = np.eye(N)
    for i in range(N-1):
        
        j = arg_maxmodule(u, i)
        
        Pi = np.eye(N)
        Pi[i, i] = Pi[j, j] = 0
        Pi[i, j] = Pi[j, i] = 1
        P = Pi @ P
        
        u = Pi @ u
        
        lam = np.eye(N)
        gamma = u[i+1:, i] / u[i, i]
        lam[i+1:, i] = -gamma
        u = lam @ u
        
        lam[i+1:, i] = gamma
        L = L @ np.matrix.transpose(Pi) @ lam          
    
    return P @ L, u, P

### <ВСПОМОГАТЕЛЬНАЯ ФУНКЦИЯ>
### Находит максимальный и минимальный элементы столбца и 
### возвращает номер наибольшего по модулю из них
def arg_maxmodule(u, i):
    j = []
    j.append([abs(np.amax(u[i:, i])), np.argmax(u[i:, i])+i])
    j.append([abs(np.amin(u[i:, i])), np.argmin(u[i:, i])+i])
    if j[0][0] > j[1][0]:
        return j[0][1]
    else: 
        return int(j[1][1])

def lup(a):
    L, u, P = diy_lu_modified(a)
    print('<=LUP recotstruction of matrix=>')
    print('Initial matrix')
    print(a)
    print('')
    print('Upper triangular matrix')
    print(u)
    print('')
    print('Lower triangular matrix')
    print(L)
    print('')
    print('Permutation matrix')
    print(P)
    print('')
    print('Reconstructed matrix')
    print(lup_reconstruct(L, u, P))
    print('')
    print('Difference between initial and reconstructed matrices')
    print(lup_reconstruct(L, u, P)-a)
    print('')
    print('')

lup(a)
lup(a1)

<=LUP recotstruction of matrix=>
Initial matrix
[[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]

Upper triangular matrix
[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -3.506e-01 -5.786e-01 -7.330e-01 -8.438e-01]
 [ 0.000e+00  0.000e+00  2.776e-17  2.421e-02  4.866e-02  6.961e-02]
 [ 0.000e+00  1.110e-16 -2.317e-17 -1.999e-19 -6.462e-04 -1.516e-03]
 [ 0.000e+00 -1.431e-16  6.463e-18  1.577e-19  0.000e+00  6.730e-06]]

Lower triangular matrix
[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    0.5   1.    0.    0.    0.   ]
 [1.    0.727 0.706 1.    0.    0.   ]
 [1.    0.857 0.41  0.835 1.    0.   ]
 [1.    0.941 0.178 0.426 0.789 1.   ]]

Permutation matr

In [ ]:
#Пожалуйста, не снимайте баллы за 1 час просрочки после субботы